# Notebook for Splunk Machine Learning Toolkit Container for TensorFlow

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk Machine Learning Toolkit (MLTK) Container for TensorFlow. This script contains an example of how to run an entity extraction algorithm over text using the spacy library.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import spacy
import sys

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("spacy version: " + spacy.__version__)

numpy version: 1.16.4
pandas version: 0.25.1
spacy version: 2.2.4


In [3]:
#import sys
#!{sys.executable} -m spacy download en_core_web_sm

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup fy21_basic | rename "本日の感想やご意見をお聞かせください" as comment, "本日のワークショップにご満足いただけましたか？" as satisfy, "ワークショップ実施日" as date| where isnotnull('comment') | eval workshop = if(date = "7/17/2020","AWS Security","Basic") | append [| inputlookup fy21_premium | rename "本日の感想や率直なご意見をお聞かせください" as comment, "本日のワークショップにご満足いただけましたか？" as satisfy, "ワークショップ実施日" as date , "本日のワークショップ種類" as workshop| where isnotnull('comment') ] | table workshop date satisfy comment | fit MLTKContainer mode=stage algo=spacy_ginza_word2vec comment from date satisfy into ws_spacy_word2vec_stage

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("spacy_entity_extraction_model in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [5]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("ws_spacy_word2vec_stage")
print(df)
print(df.shape)
print(str(param))

          date  satisfy                                            comment
0    2/14/2020        4                             本格的に使うとなるとやはり難しそうではある。
1    2/14/2020        5                  Splunkの基本動作の勉強になりました。ありがとうございました。
2    2/14/2020        5  知らないコマンドを知れただけで有益でした。\nsplunkはもっとたくさん活用できる可能性が...
3    2/14/2020        5                                分かりやすかったです。まず使ってみます
4    2/14/2020        5  事前にsplunkを少しでも触っていると飲み込みやすい内容だった。splunkの特長を強調し...
..         ...      ...                                                ...
343  8/28/2020        4              セキュリティは現在の主業務ではありませんがログ分析の方法は参考になりました
344  8/28/2020        4    途中参加でしたが、途中からでもログソースや各ログ内のフィールド、値がわかって参考になりました。
345  8/28/2020        5             実践的な事案でのハンズオンで、大変勉強になりました。ありがとうございました。
346  8/28/2020        4  自分にはPowerPoint内の課題の難易度が高かったですが、Office365分析における...
347  8/28/2020        5  他社のSIEM製品を使用した運用を実施しているが、Splunkについては未経験の中スケジュー...

[348 rows x 3 columns]
(348, 3)
{'options': {'params': {'mode': 'stage', 'algo': 'spacy_ginza_word2

## Stage 2 - create and initialize a model

In [6]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Load English tokenizer, tagger, parser, NER and word vectors
    #import en_core_web_sm
    #model = en_core_web_sm.load()
    #model = spacy.load("en_core_web_sm")
    model = spacy.load("ja_ginza")
    return model

In [7]:
model = init(df,param)

## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained (the en_core_web_sm library comes pre-packaged by spacy) and therefore this stage is a placeholder only

In [8]:
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    
    return returns

## Stage 4 - apply the model

In [9]:
def apply(model,df,param):
    X = df['comment'].values.tolist()
    
    returns = list()
    
    for i in range(len(X)):
        doc = model(str(X[i]))
        
        vec = doc.vector
        returns.append(vec)
        
    return returns

In [10]:
returns = apply(model,df,param)
returns[:5]

[array([-0.01745225, -0.17404637,  0.586339  , -0.20645565, -0.45512658,
        -0.7743225 ,  0.01516116,  0.4669725 ,  1.152631  , -0.26280355,
        -0.44293675, -0.33399585,  0.30537373,  0.34298804,  0.20133083,
        -0.15832189, -0.25830057, -0.19467053,  0.5354607 ,  0.40928903,
         0.02717442,  0.02406411, -0.4978748 , -0.2059672 , -0.45008305,
        -0.5953935 ,  0.52704895,  0.8005702 ,  1.446021  ,  0.41330144,
         0.62747097,  0.79453814, -0.23842365, -0.28951532, -0.7480491 ,
         0.90363944, -0.2587626 , -0.45700854, -0.01709555, -0.4804917 ,
        -0.02713005,  0.10595898, -0.69801325,  0.05026114, -0.09356131,
         0.22575544,  0.9437057 ,  0.2141324 , -0.51864135,  0.3103437 ,
         0.7937176 , -0.04258128,  0.2920567 , -0.62381774,  0.680776  ,
        -0.02972949,  0.3205241 ,  0.018604  , -0.32513997,  0.00753968,
        -0.14604221, -0.48977333,  0.50503415, -0.36163425,  0.07766759,
        -0.27951413, -0.09002935, -0.24485509, -0.2

## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 6 - load the model

In [12]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 7 - provide a summary of the model

In [13]:
# return model summary
def summary(model=None):
    returns = {"version": {"spacy": spacy.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code